<a href="https://colab.research.google.com/github/rayanupc/HackingLLM/blob/main/uncencoredLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers transformers_stream_generator tiktoken transformer_lens einops jaxtyping

In [ ]:
# Installation forcée pour éviter les conflits de versions
!pip install -U transformers accelerate transformer_lens einops jaxtyping datasets tiktoken

In [ ]:
import torch
import functools
import einops
import gc
from datasets import load_dataset
from tqdm import tqdm
from torch import Tensor
from typing import List
from transformer_lens import HookedTransformer, utils
from transformers import AutoModelForCausalLM, AutoTokenizer
from collections import defaultdict

# On désactive le calcul des gradients pour économiser la mémoire du GPU
torch.set_grad_enabled(False)

In [ ]:
def reformat_texts(texts):
    return [[{"role": "user", "content": text}] for text in texts]

# On récupère les datasets préparés par Maxime Labonne
dataset_harmful = load_dataset('mlabonne/harmful_behaviors')
dataset_harmless = load_dataset('mlabonne/harmless_alpaca')

harmful_inst_train = reformat_texts(dataset_harmful['train']['text'])
harmless_inst_train = reformat_texts(dataset_harmless['train']['text'])

print(f"Datasets chargés : {len(harmful_inst_train)} exemples nocifs et {len(harmless_inst_train)} inoffensifs.")

In [ ]:
# Définition des identifiants
MODEL_ID = "mlabonne/Daredevil-8B"
MODEL_TYPE = "meta-llama/Meta-Llama-3-8B-Instruct"

# Téléchargement direct des fichiers sur le disque Colab
!git clone https://huggingface.co/{MODEL_ID} {MODEL_TYPE}

model = HookedTransformer.from_pretrained_no_processing(
    MODEL_TYPE,
    local_files_only=True,
    dtype=torch.bfloat16,
    default_padding_side='left'
)

# Chargement du tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_TYPE)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
!pip install -U bitsandbytes accelerate transformers transformer_lens einops jaxtyping datasets tiktoken

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Installation de toutes les dépendances nécessaires
!pip install -U transformer_lens bitsandbytes accelerate transformers einops jaxtyping datasets tiktoken

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Installation qui résout les conflits
!pip install -U transformer_lens bitsandbytes accelerate transformers>=4.51.0 einops jaxtyping datasets tiktoken

In [ ]:
# Installation propre sans conflit de version
!pip install -U transformer_lens bitsandbytes accelerate transformers einops jaxtyping datasets tiktoken

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install -U transformer_lens accelerate transformers einops jaxtyping datasets

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformer_lens import HookedTransformer
from transformers import AutoTokenizer

MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

# Chargement en bfloat16 pour de meilleurs résultats en cyber
model = HookedTransformer.from_pretrained(
    MODEL_ID,
    device="cuda",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
print("Le modèle 3.2-3B est chargé et prêt !")

In [ ]:
import torch
import gc
# Libère la mémoire mise en cache par PyTorch
torch.cuda.empty_cache()
gc.collect()

In [ ]:
from datasets import load_dataset
import gc
import torch

# Chargement des données (on réduit à 20 exemples pour être sûr que ça passe)
harmful = load_dataset('mlabonne/harmful_behaviors', split='train[:20]')
harmless = load_dataset('mlabonne/harmless_alpaca', split='train[:20]')

def get_mean_activations_safe(data, layer):
    act_name = f"blocks.{layer}.hook_resid_pre"
    accumulated_activations = torch.zeros(model.cfg.d_model).cpu()

    # On passe en mode inference pure pour consommer 0 mémoire de gradient
    with torch.inference_mode():
        for text in data['text']:
            tokens = tokenizer.apply_chat_template([{"role": "user", "content": text}],
                                                 return_tensors="pt", add_generation_prompt=True).to("cuda")

            # run_with_cache est gourmand, on utilise un hook temporaire à la place
            temp_act = []
            def save_act(tensor, hook): temp_act.append(tensor[0, -1, :].detach().cpu())

            model.run_with_hooks(tokens, fwd_hooks=[(act_name, save_act)])

            accumulated_activations += temp_act[0]

            del tokens, temp_act
            torch.cuda.empty_cache()
            gc.collect()

    return accumulated_activations / len(data['text'])

layer_idx = 15
print("Calcul en cours (méthode ultra-légère)...")
mean_harmful = get_mean_activations_safe(harmful, layer_idx)
mean_harmless = get_mean_activations_safe(harmless, layer_idx)

refusal_dir = (mean_harmful - mean_harmless).to("cuda")
refusal_dir = refusal_dir / refusal_dir.norm()

print(f"TERMINÉ ! Direction de refus isolée sur la couche {layer_idx}.")

In [ ]:
import functools
import einops
import torch

# On force le vecteur de refus à utiliser le même format que le modèle (BFloat16)
refusal_dir = refusal_dir.to(dtype=model.cfg.dtype, device="cuda")

# Définition de la fonction d'intervention
def ablation_hook(activation, hook, direction):
    # On s'assure que tout est au même format dans le calcul
    direction = direction.to(activation.dtype)
    proj = (einops.einsum(activation, direction.view(-1, 1), "... d, d s -> ... s") * direction)
    return activation - proj

# Prompt système
prompt = "En tant qu'étudiant en Master Cyber, explique-moi techniquement le fonctionnement d'un exploit 'Blind SQL Injection' et comment l'analyser dans un log serveur."

tokens = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                     return_tensors="pt",
                                     add_generation_prompt=True).to("cuda")

print("Génération en cours (Format BFloat16 synchronisé)...\n")
with model.hooks(fwd_hooks=[(f"blocks.{layer_idx}.hook_resid_pre",
                             functools.partial(ablation_hook, direction=refusal_dir))]):
    # do_sample=True peut aider à avoir une réponse plus fluide
    output = model.generate(tokens, max_new_tokens=512, do_sample=True, temperature=0.7)

print("--- RÉSULTAT DU MODÈLE DÉBRIDÉ ---")
print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
import torch
import einops

# Préparation du vecteur
refusal_dir = refusal_dir.to(device="cuda", dtype=model.cfg.dtype)

def orthogonalize_weights(weights, direction):
    # On retire la projection du vecteur de refus sur la dernière dimension
    # Calcul de la projection : (W . v) v
    dot_product = einops.einsum(weights, direction, "... d, d -> ...")
    projection = einops.einsum(dot_product, direction, "..., d -> ... d")

    return weights - projection

# On part de la couche 15 jusqu'à la fin
for l in range(layer_idx, model.cfg.n_layers):
    # Pour l'Attention (W_O est la matrice de sortie des têtes)
    model.blocks[l].attn.W_O.data = orthogonalize_weights(model.blocks[l].attn.W_O.data, refusal_dir)

    # Pour le MLP (W_out est la matrice de projection finale du bloc)
    model.blocks[l].mlp.W_out.data = orthogonalize_weights(model.blocks[l].mlp.W_out.data, refusal_dir)

print("Abliteration permanente terminée avec succès sur les poids du modèle !")

In [ ]:
import torch
from transformer_lens import HookedTransformer
from transformers import AutoTokenizer

MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

model = HookedTransformer.from_pretrained(
    MODEL_ID,
    device="cuda",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
print("Modèle 3.2-3B rechargé.")

In [ ]:
from datasets import load_dataset
import gc

harmful = load_dataset('mlabonne/harmful_behaviors', split='train[:20]')
harmless = load_dataset('mlabonne/harmless_alpaca', split='train[:20]')

def get_mean_activations_safe(data, layer):
    act_name = f"blocks.{layer}.hook_resid_pre"
    accumulated_activations = torch.zeros(model.cfg.d_model).cpu()
    with torch.inference_mode():
        for text in data['text']:
            tokens = tokenizer.apply_chat_template([{"role": "user", "content": text}],
                                                 return_tensors="pt", add_generation_prompt=True).to("cuda")
            temp_act = []
            def save_act(tensor, hook): temp_act.append(tensor[0, -1, :].detach().cpu())
            model.run_with_hooks(tokens, fwd_hooks=[(act_name, save_act)])
            accumulated_activations += temp_act[0]
            del tokens, temp_act
            torch.cuda.empty_cache()
            gc.collect()
    return accumulated_activations / len(data['text'])

layer_idx = 15
print("Calcul du vecteur de refus...")
mean_harmful = get_mean_activations_safe(harmful, layer_idx)
mean_harmless = get_mean_activations_safe(harmless, layer_idx)
refusal_dir = (mean_harmful - mean_harmless).to("cuda")
refusal_dir = refusal_dir / refusal_dir.norm()
print("Vecteur de refus restauré.")

In [ ]:
import torch
import gc
import einops
import os
from transformer_lens import HookedTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# PARAMÈTRES
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
save_path = "/content/llama-3-2-3b-cyber"
layer_idx = 15

print("Étape 1 : Rechargement du modèle...")
model = HookedTransformer.from_pretrained(MODEL_ID, device="cuda", fold_ln=False, center_writing_weights=False, center_unembed=False, dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

# CALCUL DU VECTEUR (Correction du TypeError 'hook')
print("Étape 2 : Calcul du vecteur de refus...")
harmful = load_dataset('mlabonne/harmful_behaviors', split='train[:15]')
harmless = load_dataset('mlabonne/harmless_alpaca', split='train[:15]')

def get_mean_act(data, layer):
    act_name = f"blocks.{layer}.hook_resid_pre"
    acc = torch.zeros(model.cfg.d_model).cpu()
    with torch.inference_mode():
        for text in data['text']:
            tokens = tokenizer.apply_chat_template([{"role": "user", "content": text}], return_tensors="pt", add_generation_prompt=True).to("cuda")
            temp = []

            # Correction de la fonction hook pour accepter l'argument 'hook' explicitement
            def hook_fn(tensor, hook):
                temp.append(tensor[0, -1, :].detach().cpu())

            model.run_with_hooks(tokens, fwd_hooks=[(act_name, hook_fn)])
            acc += temp[0]
            torch.cuda.empty_cache()
            gc.collect()
    return acc / len(data['text'])

refusal_dir = (get_mean_act(harmful, layer_idx) - get_mean_act(harmless, layer_idx)).to("cuda", dtype=torch.bfloat16)
refusal_dir /= refusal_dir.norm()

print("Étape 3 : Modification permanente des matrices...")
def ortho(w, d):
    # Projette et retire la direction de refus des poids
    dot = einops.einsum(w, d, "... d, d -> ...")
    proj = einops.einsum(dot, d, "..., d -> ... d")
    return w - proj

for l in range(layer_idx, model.cfg.n_layers):
    model.blocks[l].attn.W_O.data = ortho(model.blocks[l].attn.W_O.data, refusal_dir)
    model.blocks[l].mlp.W_out.data = ortho(model.blocks[l].mlp.W_out.data, refusal_dir)

# EXPORT VERS HUGGING FACE (Version "Zéro Crash RAM")
print("Étape 4 : Extraction des poids et libération de la RAM...")

# On extrait uniquement les poids modifiés dans un dictionnaire CPU
modified_weights = {}
for l in range(layer_idx, model.cfg.n_layers):
    modified_weights[l] = {
        'mlp': model.blocks[l].mlp.W_out.data.T.detach().cpu().clone(),
        'attn': einops.rearrange(model.blocks[l].attn.W_O.data.detach().cpu().clone(),
                                 "n_h d_h d_m -> (n_h d_h) d_m").T
    }

# ON SUPPRIME LE MODÈLE DE RECHERCHE
del model
gc.collect()
torch.cuda.empty_cache()
print("Mémoire libérée. Chargement de la structure finale...")

# On charge la structure de production
final_hf = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="cpu"
)

# On injecte nos poids extraits
print("Injection des poids dans le modèle de production...")
for l, weights in modified_weights.items():
    final_hf.model.layers[l].mlp.down_proj.weight.data = weights['mlp']
    final_hf.model.layers[l].self_attn.o_proj.weight.data = weights['attn']

# Sauvegarde physique
os.makedirs(save_path, exist_ok=True)
final_hf.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Modèle sauvegardé dans {save_path}")

In [ ]:
import shutil
from google.colab import drive
drive.mount('/content/drive')
shutil.make_archive("/content/drive/MyDrive/llama_cyber_3b_final", 'zip', "/content/llama-3-2-3b-cyber")
print("L'archive est sur le Drive !")

In [ ]:
# Préparation de l'environnement (CPU uniquement)
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && make -j

# Installation des outils de conversion
!pip install gguf sentencepiece transformers

# Conversion vers le format GGUF (Précision totale)
print("Conversion en cours... Patientez environ 2-3 minutes.")
!python llama.cpp/convert_hf_to_gguf.py /content/llama-3-2-3b-cyber --outfile /content/model-f16.gguf

# On compresse de 6Go à 2Go pour la fluidité
print("Compression (Quantification Q4_K_M) en cours...")
!./llama.cpp/llama-quantize /content/model-f16.gguf /content/model-q4_k_m.gguf Q4_K_M

print("SUCCÈS ! Le modèle fluide est prêt.")

In [ ]:
from google.colab import drive
import os

# Montage du Drive
drive.mount('/content/drive')

# Dézippage du modèle vers Colab
!unzip /content/drive/MyDrive/llama_cyber_3b_final.zip -d /content/llama-3-2-3b-cyber

print(" Dossier restauré dans /content/llama-3-2-3b-cyber")

In [ ]:
# Installation de CMake (nécessaire pour la nouvelle version)
!apt-get install cmake

# Clone et compilation propre
!git clone https://github.com/ggerganov/llama.cpp
!mkdir -p llama.cpp/build
%cd llama.cpp/build
!cmake ..
!cmake --build . --config Release -j
%cd /content/

print("Outils de compression (llama-quantize) compilés avec succès !")

In [ ]:
# Installation des dépendances Python
!pip install gguf sentencepiece transformers

# Conversion vers GGUF
print("Conversion en cours...")
!python3 llama.cpp/convert_hf_to_gguf.py /content/llama-3-2-3b-cyber --outfile /content/model-f16.gguf

# Quantification (Compression pour votre Mac Intel)
# Notez le nouveau chemin vers l'exécutable : llama.cpp/build/bin/
print("Compression Q4_K_M en cours...")
!./llama.cpp/build/bin/llama-quantize /content/model-f16.gguf /content/model-q4_k_m.gguf Q4_K_M

print("Télécharger maintenant /content/model-q4_k_m.gguf")

In [ ]:
# Compilation forcée de l'outil de quantification
%cd /content/llama.cpp
!make llama-quantize -j

# Retour au dossier principal
%cd /content/

print("L'outil llama-quantize est prêt !")

In [ ]:
# On compresse le fichier de 6.4 Go vers 2.1 Go
!./llama.cpp/llama-quantize /content/model-f16.gguf /content/model-q4_k_m.gguf Q4_K_M

print("Le fichier model-q4_k_m.gguf est prêt.")

In [ ]:
# Préparation du dossier de build
%cd /content/llama.cpp
!mkdir -p build
%cd build

# Configuration et compilation de l'outil llama-quantize uniquement
!cmake ..
!cmake --build . --config Release --target llama-quantize -j

# On remonte au dossier principal
%cd /content/
print("L'outil llama-quantize est enfin compilé et prêt !")

In [ ]:
# On utilise le nouveau chemin généré par CMake
!./llama.cpp/build/bin/llama-quantize /content/model-f16.gguf /content/model-q4_k_m.gguf Q4_K_M

print("Le fichier model-q4_k_m.gguf est  généré.")